In [1]:
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitModel, SetFitTrainer
import pandas as pd
import datasets
import pyarrow as pa

# train pairs 4
train_dataframe = pd.read_csv("../data_ready/few_shot/train_pairs_random_4.csv").sample(frac=1)
eval_dataframe = pd.read_csv("../data_ready/unused_pairs_for_test_data.csv").sample(frac=1)

train_dataset = datasets.Dataset(pa.Table.from_pandas(train_dataframe))
test_dataset = datasets.Dataset(pa.Table.from_pandas(eval_dataframe))

# Load a SetFit model from Hub
model = SetFitModel.from_pretrained("facebook/bart-large-mnli")

# Create trainer
trainer = SetFitTrainer(
  model=model,
  train_dataset=train_dataset,
  eval_dataset=test_dataset,
  loss_class=CosineSimilarityLoss,
  metric="accuracy",
  batch_size=16,
  num_iterations=20, # The number of text pairs to generate for contrastive learning
  num_epochs=1, # The number of epochs to use for contrastive learning
  column_mapping={"text": "text", "class": "label"} # Map dataset columns to text/label expected by trainer
)

# Train and evaluate
trainer.train()


################ hugging face hub?? ##################
# Push model to the Hub
# trainer.push_to_hub("my-awesome-setfit-model")

# Download from Hub and run inference
# model = SetFitModel.from_pretrained("lewtun/my-awesome-setfit-model")
###############################################

# Run inference
# preds = model(["i loved the spiderman movie!", "pineapple on pizza is the worst 🤮"])

/home/blake/miniconda3/envs/tf/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-02-01 18:24:32.289116: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-01 18:24:34.979590: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.5/lib64:/usr/local/cuda-11.5/lib64
2023-02-01 18:24:34.979730: W tensorflow/compiler/xla/stream_executor/platform/defa

KeyboardInterrupt: 

In [ ]:
metrics = trainer.evaluate()